In [ ]:
import os
import numpy as np
import quaternion
from avapi.kitti import KittiObjectDataset, KittiRawDataset, KittiScenesManager

obj_data_dir_k = "../../data/KITTI/object"
raw_data_dir_k = "../../data/KITTI/raw"

KOD = KittiObjectDataset
KRD = KittiRawDataset(raw_data_dir_k)
KSM = KittiScenesManager(obj_data_dir_k)

## Test Visualization

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes, show_lidar_on_image


# Single frame visualization
KDM = KSM.get_scene_dataset_by_index(0)
frame = KDM.frames[30]
img = KDM.get_image(frame, sensor="image-2")
pc = KDM.get_lidar(frame)
objects = KDM.get_objects(frame)
show_image_with_boxes(img, objects, inline=True)
show_lidar_on_image(pc, img)

## Test Detection

In [ ]:
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import MMDetObjectDetector3D


# get models
def get_model(model: str):
    if model == "2d-img":
        M = MMDetObjectDetector2D(model="fasterrcnn", dataset="kitti", gpu=1)
    elif model == "3d-img":
        M = MMDetObjectDetector3D(model="pgd", dataset="kitti", gpu=0)
    elif model == "3d-lidar":
        M = MMDetObjectDetector3D(model="pointpillars", dataset="kitti", gpu=1)
    else:
        raise NotImplementedError(model)
    return M

In [ ]:
# get data
frame = KDM.frames[30]
img = KDM.get_image(frame, sensor="image-2")
pc = KDM.get_lidar(frame)
objects = KDM.get_objects(frame)

# get model and run inference
model = "2d-img"
M = get_model(model=model)
outputs = M(pc if "lidar" in model else img)
show_image_with_boxes(img, outputs.data, inline=True)